In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('arabicdeepfakedetection5.csv')

In [3]:
df

,Unnamed: 0,Inputs,Label
0,34977,برنامج Essentials Pro 9. 0. 2. 1. 6 Portable ي...,0
1,46463,عرب بت - أعلنت اللجنة المنظمة لمهرجان التسوق ا...,0
2,36231,وكانت وزارة الاستثمار قد رفعت أسعار تلك الخدما...,0
3,71,قال وزير الخارجية البريطاني ، ويليام هيج ، الي...,0
4,42859,حجز تشيلسي فرصة التتويج بلقب دوري أبطال أوروبا...,0
...,...,...,...
87447,99995,رائع وموقعه مميز . موقع الفندق بالقرب من الخور...,1
87448,99996,رحلة مميزة بالقرب من الحرم . تعامل الموظفين كا...,1
87449,99997,حجز لثلاث اشخاص . تسجيل الدخول للفندق و تسجيل ...,1
87450,99998,مخيب للأمل. . تسجيل الدخول استغرق اكثر من ساعتين,1


In [4]:
df.Label.value_counts()

0    43726
1    43726
Name: Label, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

text = df['Inputs']
label = df['Label']

X_train, X_val, y_train, y_val = train_test_split(text, label, test_size=0.25, stratify=label)

In [7]:
from arabert.preprocess import ArabertPreprocessor
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")

# Initialize the GPT-2 model for Arabic

gpt2_model = AutoModel.from_pretrained("akhooli/gpt2-small-arabic")

Some weights of the model checkpoint at akhooli/gpt2-small-arabic were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def tokenize_data(tokenizer, texts):
  """Tokenizes a list of text strings using the given BERT tokenizer.

  Args:
    tokenizer: A BERT tokenizer.
    texts: A list of text strings.

  Returns:
    A list of tokenized sentences, where each sentence is a list of token IDs.
  """

  tokenized_texts = []
  for text in texts:
    # Fix the error by removing the `return_tensors='pt'` argument.
    tokens = tokenizer(text, padding=True, truncation=True, max_length=128)['input_ids']
    tokenized_texts.append(tokens)

  return tokenized_texts

In [9]:
tokenizer.pad_token=True

In [10]:
x =tokenize_data(tokenizer, X_train)

In [11]:
x1 =tokenize_data(tokenizer, X_val)

In [12]:
import keras
import transformers
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
x_train=pad_sequences(x)
x_val=pad_sequences(x1)

In [13]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
# Create an embedding layer using GPT-2's embedding weights
embedding_layer = Embedding(
    input_dim=gpt2_model.config.vocab_size,
    output_dim=gpt2_model.config.hidden_size,
    weights=[gpt2_model.get_input_embeddings().weight.detach()],
    trainable=True
)

In [67]:
# Define and compile your Keras model with the pre-built GPT-2 embedding layer
model = Sequential()
model.add(embedding_layer)  # Use the pre-built embedding layer
model.add(Bidirectional(LSTM(128, return_sequences=True)))  # Bidirectional LSTM layer with 128 units
model.add(Dropout(0.2))  # Apply dropout for regularization
model.add(Bidirectional(LSTM(64)))  # Bidirectional LSTM layer with 64 units
model.add(Dropout(0.2))  # Apply dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train your model using X_train and y_train
# Evaluate your model on X_val and y_val

In [68]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],)


In [69]:
model.fit(x_train, y_train, epochs=10,steps_per_epoch=100,batch_size=32)


Epoch 1/10
100/100 [==============================] - 8s 56ms/step - loss: 0.2373 - accuracy: 0.8916
Epoch 2/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0391 - accuracy: 0.9881
Epoch 3/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0275 - accuracy: 0.9925
Epoch 4/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0425 - accuracy: 0.9844
Epoch 5/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0255 - accuracy: 0.9925
Epoch 6/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0245 - accuracy: 0.9900
Epoch 7/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0213 - accuracy: 0.9928
Epoch 8/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0158 - accuracy: 0.9956
Epoch 9/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0195 - accuracy: 0.9931
Epoch 10/10
100/100 [==============================] - 6s 56ms/step - loss: 0.0108 - accura

In [70]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 3s 23ms/step - loss: 0.0191 - accuracy: 0.9948


[0.01909259334206581, 0.994785726070404]

In [71]:
x_train.shape[1]

128

In [98]:
import numpy as np
text=''



text_p = [text]
x2 = tokenize_data(gpt2_tokenizer, text_p)

# Make a prediction
prediction_input = np.array(x2).reshape(-1)
prediction_input=pad_sequences([prediction_input],128)


output = model.predict(prediction_input)




if output > 0.5:
    predicted_class = 'human written'
else:
    predicted_class = 'machine generated'

# Print the predicted class
print('Predicted class:', predicted_class,output)


1/1 [==============================] - 0s 18ms/step
Predicted class: human written [[0.58353674]]


In [99]:
model.save('aragpt2deepfaketextdetection.h5')

In [14]:
# Define and compile your Keras model with the pre-built GPT-2 embedding layer
model = Sequential()
model.add(embedding_layer)  # Use the pre-built embedding layer

model.add(LSTM(12))  # Bidirectional LSTM layer with 64 units
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train your model using X_train and y_train
# Evaluate your model on X_val and y_val

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],)


In [16]:
model.fit(x_train, y_train, epochs=10,steps_per_epoch=100,batch_size=32)


Epoch 1/10
100/100 [==============================] - 11s 41ms/step - loss: 0.2777 - accuracy: 0.9041
Epoch 2/10
100/100 [==============================] - 4s 41ms/step - loss: 0.0553 - accuracy: 0.9872
Epoch 3/10
100/100 [==============================] - 6s 61ms/step - loss: 0.0409 - accuracy: 0.9887
Epoch 4/10
100/100 [==============================] - 6s 62ms/step - loss: 0.0340 - accuracy: 0.9912
Epoch 5/10
100/100 [==============================] - 4s 40ms/step - loss: 0.0322 - accuracy: 0.9912
Epoch 6/10
100/100 [==============================] - 4s 41ms/step - loss: 0.0283 - accuracy: 0.9928
Epoch 7/10
100/100 [==============================] - 6s 62ms/step - loss: 0.0214 - accuracy: 0.9941
Epoch 8/10
100/100 [==============================] - 4s 45ms/step - loss: 0.0308 - accuracy: 0.9897
Epoch 9/10
100/100 [==============================] - 4s 44ms/step - loss: 0.0167 - accuracy: 0.9962
Epoch 10/10
100/100 [==============================] - 4s 44ms/step - loss: 0.0181 - accur

In [17]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 2s 12ms/step - loss: 0.0255 - accuracy: 0.9933


[0.02554127760231495, 0.9933220744132996]

In [18]:
model.save('aragpt2deepfaketextdetectionLSTM.h5')